# LHS 712 assignment 3
#### Ella Li

# 1. CRF

### Read data

In [1]:
# !pip install --force-reinstall --no-dependencies "scikit-learn==0.24.2"

In [2]:
# !pip install seqeval

In [3]:
import string

In [4]:
def read_file(f):
    data = open(f,'r').readlines()[1:]
    row_id = [i.split('\t')[0].strip() for i in data]
    data = [i.split('\t')[1].strip().split(' ') for i in data]
    return row_id,data

In [5]:
row_id_text, texts = read_file('./review_data/REVIEW_TEXT.txt')
row_id_tags, tags = read_file('./review_data/REVIEW_LABELSEQ.txt')





### Two entities of interest -- AE (adverse events) and SSI (signs, symptoms, and indications).

We use BIO scheme: 

     B- to denote beginning of a tagged named entity, 
     
     I- to denote inside a tagged named entity tag, 
     
     O to denote outside of any tagged named entity 
     
So, your sequential labeling task has five tags: B-AE, I-AE, B-SSI, I-SSI, and O.

In [6]:
index = 5
print('num of data', len(row_id_text))
assert len(row_id_text) == len(row_id_tags)
######## label dataset 

print('-'*89)
print('Token\tTag')
for idx in range(len(texts[index])):
    print(texts[index][idx], '\t', tags[index][idx])

num of data 4744
-----------------------------------------------------------------------------------------
Token	Tag
I 	 O
had 	 O
terrible 	 B-AE
anxiety 	 I-AE
the 	 I-AE
whole 	 I-AE
time 	 I-AE
, 	 O
the 	 B-AE
worst 	 I-AE
kind 	 I-AE
of 	 I-AE
anxiety 	 I-AE
I've 	 O
ever 	 O
experienced. 	 O


In [7]:
index = 22
print('Token\tTag')
for idx in range(len(texts[index])):
    print(texts[index][idx], '\t', tags[index][idx])

Token	Tag
constipation 	 B-AE
, 	 O
drastic 	 B-AE
mood 	 I-AE
swings 	 I-AE
, 	 O
100% 	 O
helped 	 O
my 	 O
anxiety 	 B-SSI
and 	 O
panic 	 B-SSI
. 	 O


### Inputs

In [8]:
def word2features(word):
    features = {
        'word.lower()': word.lower(),  # 
        'word.isdigit()': word.isdigit(), 
        ## you can add more feature extractor here
        # https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#features
    }
    return features

def text2features(text):
    return [word2features(i) for i in text]


In [9]:
X = [text2features(text) for text in texts]
y = tags

### Train / Validation sets

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.2, random_state=42)

### CRF model

In [11]:
# https://sklearn-crfsuite.readthedocs.io/en/latest/
from sklearn_crfsuite import CRF
crf = CRF()

### Training & Prediction

In [12]:
crf.fit(X_train, y_train) # train step 
y_pred = crf.predict(X_validation) # inference step

### Result

In [13]:
from sklearn_crfsuite.metrics import flat_classification_report
report = flat_classification_report(y_validation, y_pred)
print(report)

              precision    recall  f1-score   support

        B-AE       0.70      0.43      0.53       752
       B-SSI       0.78      0.50      0.61       168
        I-AE       0.59      0.33      0.42      1485
       I-SSI       0.21      0.05      0.08        66
           O       0.89      0.97      0.92     11859

    accuracy                           0.86     14330
   macro avg       0.63      0.45      0.51     14330
weighted avg       0.84      0.86      0.84     14330



C:\Users\Jiaqi\anaconda3\new\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [14]:
# find instruction of hyperparameters here :https://sklearn-crfsuite.readthedocs.io/en/latest/api.html

crf = CRF(algorithm='lbfgs', # Gradient descent using the L-BFGS method
    c1=0.2, # The coefficient for L1 regularization.
    c2=0.2,  # The coefficient for L1 regularization.
    max_iterations=50,
    all_possible_transitions=True)

crf.fit(X_train, y_train) # train step 
y_pred = crf.predict(X_validation) # inference step
report = flat_classification_report(y_validation, y_pred)
print(report)

              precision    recall  f1-score   support

        B-AE       0.67      0.50      0.57       752
       B-SSI       0.77      0.55      0.64       168
        I-AE       0.58      0.41      0.48      1485
       I-SSI       0.25      0.06      0.10        66
           O       0.90      0.96      0.93     11859

    accuracy                           0.87     14330
   macro avg       0.64      0.49      0.54     14330
weighted avg       0.85      0.87      0.85     14330



C:\Users\Jiaqi\anaconda3\new\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [15]:
#==============================================
# add features into crf

def word2features(word):
    features = {
        'word': word,
        'len(word)': len(word),
        'word.lower()': word.lower(),  
        'word.upper()': word.upper(),
        'word.isdigit()': word.isdigit(), 
        ## you can add more feature extractor here
        # https://sklearn-crfsuite.readthedocs.io/en/latest/tutorial.html#features
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.ispunctuation()': (word in string.punctuation),
    }
    return features

def text2features(text):
    return [word2features(i) for i in text]


In [16]:
# x is features and y is tags

X = [text2features(text) for text in texts]
y = tags

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size = 0.2, random_state=42)

In [18]:
crf = CRF(algorithm='lbfgs', # Gradient descent using the L-BFGS method
    c1=0.2, # The coefficient for L1 regularization.
    c2=0.2,  # The coefficient for L1 regularization.
    max_iterations=50,
    all_possible_transitions=True)

crf.fit(X_train, y_train) # train step 
y_pred = crf.predict(X_validation) # inference step
report = flat_classification_report(y_validation, y_pred)
print(report)

              precision    recall  f1-score   support

        B-AE       0.70      0.53      0.60       752
       B-SSI       0.74      0.55      0.63       168
        I-AE       0.66      0.44      0.53      1485
       I-SSI       0.14      0.05      0.07        66
           O       0.91      0.97      0.94     11859

    accuracy                           0.88     14330
   macro avg       0.63      0.51      0.55     14330
weighted avg       0.86      0.88      0.87     14330



C:\Users\Jiaqi\anaconda3\new\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


* we can see that the precision, recall, and F1-score metrics vary considerably across the different named entity classes. The "O" class (which stands for "other" or "not a named entity") has very high precision, recall, and F1-score, indicating that it is relatively easy to identify instances that do not belong to any named entity class. On the other hand, the "I-SSI" class has very low precision, recall, and F1-score, indicating that it is difficult to correctly identify instances that belong to this class. 
* The overall accuracy of the model is 0.88, which means that it correctly identified the named entity class for 88% of the instances in the validation set, which increased a little than the baseline 0.86 and 0.87.

In [19]:
# tune parameters

# crf = CRF(algorithm='lbfgs',
#           c1=0.1,
#           c2=0.1,
#           max_iterations=100,
#           all_possible_transitions=True,
#           min_freq=5,
#           epsilon=0.01)

# crf.fit(X_train, y_train) # train step 
# y_pred = crf.predict(X_validation) # inference step
# report = flat_classification_report(y_validation, y_pred)
# print(report)

# 2. LSTM

In [20]:
def read_file(f):
    data = open(f,'r').readlines()[1:]
    row_id = [i.split('\t')[0].strip() for i in data]
    data = [i.split('\t')[1].strip().split(' ') for i in data]
    return row_id,data

row_id_text, texts = read_file('./review_data/REVIEW_TEXT.txt')
row_id_tags, tags = read_file('./review_data/REVIEW_LABELSEQ.txt')


In [21]:
import string
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional, Activation
from tensorflow.keras.metrics import Precision, Recall
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.preprocessing import MultiLabelBinarizer

preporcessing for LSTM:

1. Stem the words in the texts using the SnowballStemmer: Stemming is a text preprocessing technique that reduces words to their root form. By using stemming, we can combine different forms of the same word into a single term. This step helps to reduce the vocabulary size and generalizes the model. 

2. Tokenize the stemmed texts using the Keras tokenizer: Tokenization is the process of converting the text data into individual tokens (usually words). In this step, use the Keras Tokenizer class to tokenize the stemmed texts. The tokenizer also helps in building the vocabulary of the text data by assigning a unique index to each token.

3. Convert the tokenized texts into sequences: After tokenizing the text data, need to convert it into sequences of indices. These indices correspond to the tokens' positions in the vocabulary. use the texts_to_sequences method of the Keras tokenizer to convert the tokenized texts into sequences of indices.

4. Pad the sequences to have the same length: For training a neural network, it is necessary to have input data with the same length. In this step, we pad the sequences of indices with zeros to make them have the same length. we use the Keras pad_sequences function to perform this operation.

In [22]:
## preporcessing for LSTM:

# Create a list of unique words by flattening the text data and converting it into a set
unique_words = list(set(word for text in texts for word in text))

# maps every unique word to an index, adding a special padding word with index 0
word2idx = {word: i+1 for i, word in enumerate(unique_words)}
word2idx["PAD"] = 0

# map each unique tag to an index/ map each index back to its corresponding tag
unique_tags = list(set(tag for tag_seq in tags for tag in tag_seq))
label2idx = {tag: i for i, tag in enumerate(unique_tags)}
idx2label = {i: tag for tag, i in label2idx.items()}

# Pad sequences to keep length same
x = pad_sequences(maxlen=25, sequences=[[word2idx[word] for word in text] for text in texts], padding="post", value=word2idx["PAD"])
y = pad_sequences(maxlen=25, sequences=[[label2idx[tag] for tag in tag_seq] for tag_seq in tags], padding="post", value=label2idx["O"])

# Convert tag indices to one-hot encoded vectors
y = [to_categorical(tag_seq, num_classes=len(unique_tags)) for tag_seq in y]


In [23]:
# split data set

x_train, x_validation, y_train, y_validation  = train_test_split(x, y, test_size = 0.2)

In [24]:
# build LSTM Model

input_dim = len(word2idx)
output_dim = 20
input_length = 25
num_labels = len(label2idx)

model = Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))
model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
model.add(TimeDistributed(Dense(num_labels, activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 25, 20)            173680    
                                                                 
 bidirectional (Bidirectiona  (None, 25, 100)          28400     
 l)                                                              
                                                                 
 time_distributed (TimeDistr  (None, 25, 5)            505       
 ibuted)                                                         
                                                                 
Total params: 202,585
Trainable params: 202,585
Non-trainable params: 0
_________________________________________________________________


In [25]:
# train 

batch_size = 16
epochs = 8
validation_split = 0.1

training_results = model.fit(
    x_train,
    np.array(y_train),
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
)

# predict
y_pred = model.predict(x_validation)

# convert the predicted probabilities into a more interpretable format that shows the actual predicted tags for each input sequence
y_pred = np.argmax(y_pred, axis=-1)
y_validation = np.argmax(y_validation, -1)

def indices_to_labels(indices, idx2label):
    return [[idx2label[i] for i in row] for row in indices]

y_pred = indices_to_labels(y_pred, idx2label)
y_validation = indices_to_labels(y_validation, idx2label)

Epoch 1/8
214/214 [==============================] - 5s 10ms/step - loss: 0.4623 - accuracy: 0.8999 - val_loss: 0.3040 - val_accuracy: 0.9112
Epoch 2/8
214/214 [==============================] - 2s 7ms/step - loss: 0.2540 - accuracy: 0.9103 - val_loss: 0.2336 - val_accuracy: 0.9215
Epoch 3/8
214/214 [==============================] - 2s 7ms/step - loss: 0.1870 - accuracy: 0.9320 - val_loss: 0.1996 - val_accuracy: 0.9324
Epoch 4/8
214/214 [==============================] - 2s 7ms/step - loss: 0.1395 - accuracy: 0.9509 - val_loss: 0.1820 - val_accuracy: 0.9405
Epoch 5/8
214/214 [==============================] - 2s 7ms/step - loss: 0.1037 - accuracy: 0.9652 - val_loss: 0.1729 - val_accuracy: 0.9438
Epoch 6/8
214/214 [==============================] - 2s 7ms/step - loss: 0.0788 - accuracy: 0.9739 - val_loss: 0.1834 - val_accuracy: 0.9462
Epoch 7/8
214/214 [==============================] - 2s 8ms/step - loss: 0.0617 - accuracy: 0.9798 - val_loss: 0.1896 - val_accuracy: 0.9421
Epoch 8/8
30

In [26]:
report = flat_classification_report(y_pred=y_pred, y_true=y_validation)
print(report)

              precision    recall  f1-score   support

        B-AE       0.68      0.63      0.65       641
       B-SSI       0.61      0.50      0.55       183
        I-AE       0.68      0.58      0.63      1160
       I-SSI       0.28      0.16      0.20       100
           O       0.97      0.98      0.98     21641

    accuracy                           0.95     23725
   macro avg       0.64      0.57      0.60     23725
weighted avg       0.94      0.95      0.94     23725



C:\Users\Jiaqi\anaconda3\new\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


In [27]:
#############################################################################

In [28]:
#  next, let us try to make it better/ improve model

In [29]:
from keras.optimizers import Adam
from keras.layers import Activation
from keras.optimizers import RMSprop

In [30]:
# reread data
row_id_text, texts = read_file('./review_data/REVIEW_TEXT.txt')
row_id_tags, tags = read_file('./review_data/REVIEW_LABELSEQ.txt')



## preporcessing for LSTM:

# Create a list of unique words by flattening the text data and converting it into a set
unique_words = list(set(word for text in texts for word in text))

# maps every unique word to an index, adding a special padding word with index 0
word2idx = {word: i+1 for i, word in enumerate(unique_words)}
word2idx["PAD"] = 0

# map each unique tag to an index/ map each index back to its corresponding tag
unique_tags = list(set(tag for tag_seq in tags for tag in tag_seq))
label2idx = {tag: i for i, tag in enumerate(unique_tags)}
idx2label = {i: tag for tag, i in label2idx.items()}

# Pad sequences to keep length same
x = pad_sequences(maxlen=25, sequences=[[word2idx[word] for word in text] for text in texts], padding="post", value=word2idx["PAD"])
y = pad_sequences(maxlen=25, sequences=[[label2idx[tag] for tag in tag_seq] for tag_seq in tags], padding="post", value=label2idx["O"])

# Convert tag indices to one-hot encoded vectors
y = [to_categorical(tag_seq, num_classes=len(unique_tags)) for tag_seq in y]

# split data set

x_train, x_validation, y_train, y_validation  = train_test_split(x, y, test_size = 0.2)

# build LSTM Model

input_dim = len(word2idx)
output_dim = 20
input_length = 25
num_labels = len(label2idx)

model = Sequential()
model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))
model.add(Bidirectional(LSTM(units=100, return_sequences=True)))
model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
model.add(TimeDistributed(Dense(units=64, activation='relu')))
model.add(TimeDistributed(Dense(num_labels, activation='softmax')))

optimizer = RMSprop(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
model.summary()


# train 

batch_size = 16
epochs = 8
validation_split = 0.1

training_results = model.fit(
    x_train,
    np.array(y_train),
    batch_size=batch_size,
    epochs=epochs,
    validation_split=validation_split,
)

# predict
y_pred = model.predict(x_validation)

# convert the predicted probabilities into a more interpretable format that shows the actual predicted tags for each input sequence
y_pred = np.argmax(y_pred, axis=-1)
y_validation = np.argmax(y_validation, -1)

def indices_to_labels(indices, idx2label):
    return [[idx2label[i] for i in row] for row in indices]

y_pred = indices_to_labels(y_pred, idx2label)
y_validation = indices_to_labels(y_validation, idx2label)

report = flat_classification_report(y_pred=y_pred, y_true=y_validation)
print(report)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 25, 20)            173680    
                                                                 
 bidirectional_1 (Bidirectio  (None, 25, 200)          96800     
 nal)                                                            
                                                                 
 bidirectional_2 (Bidirectio  (None, 25, 100)          100400    
 nal)                                                            
                                                                 
 time_distributed_1 (TimeDis  (None, 25, 64)           6464      
 tributed)                                                       
                                                                 
 time_distributed_2 (TimeDis  (None, 25, 5)            325       
 tributed)                                            

C:\Users\Jiaqi\anaconda3\new\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=None as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  warnings.warn(f"Pass {args_msg} as keyword args. From version "


* I tried multiple times to improve the model, finally I kept these changes:
1. Adding an extra layer of Bidirectional LSTM with 100 units
2. Adding an additional layer of TimeDistributed dense with 64 units and ReLU activation
3. Changing the optimizer from Adam to RMSprop and setting a lower learning rate of 0.001


* leads to improvements:
1. B-AE: improved from 0.67 to 0.75
2. B-SSI: improved from 0.52 to 0.62
3. I-AE: improved from 0.64 to 0.68
4. I-SSI: improved from 0.15 to 0.28
5. O: stayed the same at 0.98
* Overall, the new model shows an improvement in the f1-score for all tags except for "O", which remains the same. The largest improvement is seen for the "I-SSI" tag, which goes from an f1-score of 0.15 to 0.28.

## predict/ generate test data label

In [31]:
# read

row_id_text, texts = read_file('./review_data/REVIEW_TEXT.txt')
row_id_tags, tags = read_file('./review_data/REVIEW_LABELSEQ.txt')
row_id_text_test, texts_test = read_file('./review_data/TEST_REVIEW_TEXT.txt')

In [32]:
# prepocess test data

unique_words_test = list(set(word for text in texts_test for word in text))
word2idx_test = dict(zip(unique_words_test, range(1, len(unique_words_test)+1)))
word2idx_test["PAD"] = 0

x_test = [[word2idx_test[j] for j in i] for i in texts_test]
x_test = pad_sequences(maxlen=25, sequences=x_test, padding="post", value=word2idx_test["PAD"])


In [33]:
# use the trained model to predict test data

y_pred_test = model.predict(x_test)

# convert

y_pred_test = np.argmax(y_pred_test, axis=-1)
y_pred_test = indices_to_labels(y_pred_test, idx2label)

40/40 [==============================] - 0s 9ms/step


In [34]:
# reorgnize
y_pred_test = [' '.join([label for label in sub_list]) for sub_list in y_pred_test]

In [35]:
# export

column_names = ['ID','TAGSEQ']
test_df = pd.DataFrame(zip(row_id_text_test, y_pred_test), columns=column_names)

# Write the dataframe to a tab-separated file
try:
    test_df.to_csv('TEST_DATA_LABELSEQ.txt', sep='\t', index=False)
    print("Predicted labels saved successfully.")
except Exception as e:
    print("Error while writing to file:", e)


Predicted labels saved successfully.
